In [1]:
import torch
import torch.nn as nn
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [15]:
# 0 data prepocessing

bc = datasets.load_breast_cancer()
print(bc.keys())
X, y = bc.data, bc.target

dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename', 'data_module'])


In [16]:
n_samples, n_features = X.shape
print(n_samples, n_features)

569 30


In [44]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

In [46]:
# scale, the data will have zero mean and unit variance

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

X_train = torch.from_numpy(X_train.astype(np.float32))
X_test = torch.from_numpy(X_test.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))

y_train = y_train.view(-1, 1)
y_test = y_test.view(-1, 1)

In [53]:
# 1 model
# f = wx + b, sigmoid at the end

class LogisticRegression(nn.Module):

    def __init__(self, n_input_features):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(n_input_features, 1)
    
    def forward(self, x):
        y_pred = torch.sigmoid(self.linear(x))
        return y_pred

model = LogisticRegression(n_features)

In [57]:
# 2 loss and optimizer

lr = 0.01
criterion = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr = lr)

In [64]:
# 3 training loop
num_epochs = 100

for epoch in range(num_epochs):
    # forward pass and loss
    y_pred = model(X_train)
    loss = criterion(y_pred, y_train)

    # backward pass
    loss.backward()

    # updates
    optimizer.step()

    # zero gradients
    optimizer.zero_grad()

    if (epoch+1) % 10 ==0:
        print(f'epoch: {epoch+1}, loss = {loss.item():.4f}')

epoch: 10, loss = 0.1435
epoch: 20, loss = 0.1414
epoch: 30, loss = 0.1394
epoch: 40, loss = 0.1374
epoch: 50, loss = 0.1356
epoch: 60, loss = 0.1338
epoch: 70, loss = 0.1322
epoch: 80, loss = 0.1306
epoch: 90, loss = 0.1290
epoch: 100, loss = 0.1275


In [71]:
with torch.no_grad():
    y_pred = model(X_test)
    print(y_pred[:5])
    y_pred_cls = y_pred.round()
    print(y_pred_cls[:5])
    acc = y_pred_cls.eq(y_test).sum() / float(y_test.shape[0])
    print(f'accuracy = {acc:.4f}')

tensor([[0.8734],
        [0.9945],
        [0.7003],
        [0.9613],
        [0.9419]])
tensor([[1.],
        [1.],
        [1.],
        [1.],
        [1.]])
accuracy = 0.9298
